In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.applications.efficientnet as efn
import scipy as sp
import os
from os.path import isfile, join

EFNETS = (efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
          efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7)
    
print(f'tf version: {tf.__version__}')

tf version: 2.3.1


In [2]:
N_CLASSES = 9
SEED = 17
IMAGES_DIR = '/kaggle/input/skillbox-emotions/'
TEST_DIR = IMAGES_DIR + '/test_kaggle'

train_df = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv').iloc[:, 1:]
sub = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
images_test_filenames =  [f for f in os.listdir(TEST_DIR) if isfile(join(TEST_DIR, f))]

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED)

In [3]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()

In [4]:
EFNET_NO = 2
BATCH_SIZE = 64
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
IMAGE_SIZE = IMSIZES[EFNET_NO]

train_data = train_data_gen.flow_from_dataframe(
              train_df, directory=IMAGES_DIR, x_col='image_path', y_col='emotion',
              target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='sparse', batch_size=BATCH_SIZE, shuffle=True
          )
val_data = val_data_gen.flow_from_dataframe(
              val_df, directory=IMAGES_DIR, x_col='image_path', y_col='emotion', 
              target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='sparse', batch_size=BATCH_SIZE, shuffle=False
          )
test_data = test_data_gen.flow_from_dataframe(
              sub, directory=TEST_DIR, x_col='image_path', y_col=None,  
              target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode=None, batch_size=BATCH_SIZE, shuffle=False
          )

Found 45041 validated image filenames belonging to 9 classes.
Found 5005 validated image filenames belonging to 9 classes.
Found 5000 validated image filenames.


In [5]:
model = tf.keras.Sequential([
    EFNETS[EFNET_NO](
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        weights=None,
        include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb2 (Functional)  (None, 9, 9, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dense (Dense)                (None, 9)                 12681     
Total params: 7,781,250
Trainable params: 7,713,675
Non-trainable params: 67,575
_________________________________________________________________


In [6]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [7]:
history = model.fit(
    train_data, validation_data=val_data, 
    epochs=10, callbacks=[checkpoint])

Epoch 1/10
704/704 [==============================] - ETA: 0s - loss: 2.1951 - accuracy: 0.1462
Epoch 00001: val_accuracy improved from -inf to 0.15644, saving model to model.h5
704/704 [==============================] - 832s 1s/step - loss: 2.1951 - accuracy: 0.1462 - val_loss: 2.1531 - val_accuracy: 0.1564
Epoch 2/10
704/704 [==============================] - ETA: 0s - loss: 2.1455 - accuracy: 0.1639
Epoch 00002: val_accuracy improved from 0.15644 to 0.19201, saving model to model.h5
704/704 [==============================] - 701s 996ms/step - loss: 2.1455 - accuracy: 0.1639 - val_loss: 2.1192 - val_accuracy: 0.1920
Epoch 3/10
704/704 [==============================] - ETA: 0s - loss: 1.9469 - accuracy: 0.2753
Epoch 00003: val_accuracy improved from 0.19201 to 0.32328, saving model to model.h5
704/704 [==============================] - 702s 997ms/step - loss: 1.9469 - accuracy: 0.2753 - val_loss: 1.8444 - val_accuracy: 0.3233
Epoch 4/10
704/704 [==============================] - ETA:

In [8]:
model.load_weights('model.h5')

In [9]:
mapping = {k: i for i, k in train_data.class_indices.items()}

In [10]:
test_preds = model.predict_classes(test_data)
test_preds = [mapping[i] for i in test_preds]
sub['emotion'] = test_preds
sub.to_csv('preds.csv', index=False)